In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

In [3]:
df_train.head()

,Unnamed: 0,review,score,thumbsUpCount,reviewCreatedVersion,replyContent,timeToReply
0,0,Banyak bug nya!!!! Dikit² eror terus,0,0,33.0,"Hello BANK Friend, sorry for the issues you’re...",0 days 05:06:00
1,1,Cash deposit menu does not appear Even after u...,2,0,32.0,"Hello, Friend BANK. Kindly upgrade the BANKApp...",0 days 14:25:00
2,2,Sangat membantu,4,0,33.0,"Hello BANK Customer, we appreciate you using t...",0 days 00:41:00
3,3,Very cool,4,0,33.0,"Hello BANK Friend, we appreciate your review. ...",0 days 00:30:00
4,4,Improved,4,0,33.0,"Hello BANK Friend, we appreciate your feedback...",0 days 15:05:00


In [4]:
#thumbsUpCountごとのscoreの平均値を計算
df_train.groupby("thumbsUpCount")["score"].mean()

thumbsUpCount
0      3.058113
1      2.076923
2      2.133333
3      0.666667
4      1.250000
5      1.250000
6      0.000000
7      0.000000
9      2.000000
10     4.000000
16     0.000000
19     0.000000
21     4.000000
29     0.000000
62     4.000000
72     0.000000
77     4.000000
93     1.000000
152    2.000000
167    0.000000
464    0.000000
Name: score, dtype: float64

In [5]:
#リプライに'we appricieate'が含まれているかどうか
is_we_appreciate = df_train["replyContent"].str.contains("we appreciate", na=False)
df_train["is_we_appreciate"] = is_we_appreciate

is_we_appreciate_test = df_test["replyContent"].str.contains("we appreciate", na=False)
df_test["is_we_appreciate"] = is_we_appreciate_test

In [6]:
#リプライに'Sorry'が含まれているかどうか
is_sorry = df_train["replyContent"].str.contains("sorry", na=False)
df_train["is_sorry"] = is_sorry

is_sorry_test = df_test["replyContent"].str.contains("sorry", na=False)
df_test["is_sorry"] = is_sorry_test

In [7]:
df_train.groupby("is_we_appreciate")["score"].mean()

is_we_appreciate
False    2.851256
True     3.756382
Name: score, dtype: float64

In [8]:
#reviewCreatedVersionごとに、is_we_appreciateがtrueだった時のscoreの平均値を計算
df_train.groupby(["reviewCreatedVersion", "is_we_appreciate"])["score"].mean()

reviewCreatedVersion  is_we_appreciate
0.0                   False               4.000000
1.0                   False               0.000000
2.0                   False               0.000000
3.0                   False               1.571429
                      True                4.000000
4.0                   False               0.777778
                      True                3.666667
5.0                   False               2.333333
                      True                4.000000
6.0                   False               2.200000
7.0                   False               1.333333
                      True                3.000000
8.0                   False               1.285714
9.0                   False               1.800000
10.0                  False               0.000000
11.0                  False               2.333333
                      True                4.000000
13.0                  False               0.000000
                      True                4

In [9]:
df_test.head()

,Unnamed: 0,review,thumbsUpCount,reviewCreatedVersion,replyContent,timeToReply,is_we_appreciate,is_sorry
0,8767,Mantap,0,33.0,"Hello BANK Friend, we appreciate your feedback...",0 days 02:45:00,True,False
1,8768,Straightforward and uncomplicated.,0,32.0,Thanks for the feedback. I hope the BANKApp ap...,0 days 07:30:00,False,False
2,8769,Easier to transact,0,33.0,We appreciate your feedback. Keep enhancing yo...,0 days 14:01:00,False,False
3,8770,"Alhamdulillah, the BANKApp application is very...",0,33.0,We appreciate you using the BANKApp app and en...,0 days 06:31:00,False,False
4,8771,Ioweu,0,NaN,"BANK friends, should you encounter any issues,...",0 days 22:31:00,False,False


In [10]:
df_train.head()

,Unnamed: 0,review,score,thumbsUpCount,reviewCreatedVersion,replyContent,timeToReply,is_we_appreciate,is_sorry
0,0,Banyak bug nya!!!! Dikit² eror terus,0,0,33.0,"Hello BANK Friend, sorry for the issues you’re...",0 days 05:06:00,False,True
1,1,Cash deposit menu does not appear Even after u...,2,0,32.0,"Hello, Friend BANK. Kindly upgrade the BANKApp...",0 days 14:25:00,False,False
2,2,Sangat membantu,4,0,33.0,"Hello BANK Customer, we appreciate you using t...",0 days 00:41:00,True,False
3,3,Very cool,4,0,33.0,"Hello BANK Friend, we appreciate your review. ...",0 days 00:30:00,True,False
4,4,Improved,4,0,33.0,"Hello BANK Friend, we appreciate your feedback...",0 days 15:05:00,True,False


In [11]:
def convert_to_hours(time_str):
    if isinstance(time_str, str) and " days " in time_str:
        try:
            days, time = time_str.split(" days ")
            hours, minutes, seconds = map(int, time.split(":"))
            total_hours = int(days) * 24 + hours + minutes / 60 + seconds / 3600
            return total_hours
        except ValueError:
            return np.nan  # 不正なデータの場合はNaNを返す
    else:
        return np.nan  # time_strが文字列でないか有効な形式でない場合はNaNを返す

In [12]:
df_train["total_hours"] = df_train["timeToReply"].apply(convert_to_hours)
df_test["total_hours"] = df_test["timeToReply"].apply(convert_to_hours)

In [13]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

def add_sentiment_columns(df, review_column):
    # 感情スコアを格納するリストを作成
    neg_scores = []
    neu_scores = []
    pos_scores = []
    compound_scores = []

    # 各レビューに対して感情分析を行い、スコアをリストに追加
    for review in df[review_column]:
        scores = sid.polarity_scores(review)
        neg_scores.append(scores["neg"])
        neu_scores.append(scores["neu"])
        pos_scores.append(scores["pos"])
        compound_scores.append(scores["compound"])

    # データフレームに新しい列としてスコアを追加
    df["neg"] = neg_scores
    df["neu"] = neu_scores
    df["pos"] = pos_scores
    df["compound"] = compound_scores

    return df

In [14]:
df_train = add_sentiment_columns(df_train, "review")
df_test = add_sentiment_columns(df_test, "review")

In [17]:
from datasets import load_dataset

# IMDbデータセットをロード
dataset = load_dataset("imdb")

Trying to resume download...

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /datasets/imdb/resolve/e6281661ce1c48d982bc483cf8a173c1bbeb5d31/plain_text/unsupervised-00000-of-00001.parquet (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x29a38d1d0>: Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 2e7cf4f4-5eea-414b-9a3e-22fb29586b4d)')

In [ ]:
from transformers import LlamaTokenizer

# トークナイザーのロード
tokenizer = LlamaTokenizer.from_pretrained("huggingface/llama-model")


def preprocess_data(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_dataset = dataset.map(preprocess_data, batched=True)

train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]


from transformers import LlamaForSequenceClassification, Trainer, TrainingArguments

# モデルのロード
model = LlamaForSequenceClassification.from_pretrained("huggingface/llama-model", num_labels=2)

# 訓練設定
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [15]:
from transformers import LlamaForSequenceClassification, LlamaTokenizer

# トークナイザーとモデルの読み込み
tokenizer = LlamaTokenizer.from_pretrained("your_finetuned_llama_model")
model = LlamaForSequenceClassification.from_pretrained("your_finetuned_llama_model")

/Users/iwasakimao/workspace/mufj_signate_mywork/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: 
LlamaTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


## 学習

In [20]:
X_train = df_train[["thumbsUpCount", "reviewCreatedVersion", "is_we_appreciate", "is_sorry", "total_hours", "neg", "neu", "pos", "compound"]]
y_train = df_train["score"]
X_test = df_test[["thumbsUpCount", "reviewCreatedVersion", "is_we_appreciate", "is_sorry", "total_hours", "neg", "neu", "pos", "compound"]]

In [21]:
#lightgbmを使って学習
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train)
params = {
    "objective": "regression",
    "metrics": "rmse"
}
model = lgb.train(params, lgb_train, num_boost_round=100)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000851 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1317
[LightGBM] [Info] Number of data points in the train set: 8767, number of used features: 9
[LightGBM] [Info] Start training from score 3.037299


In [22]:
X_test

,thumbsUpCount,reviewCreatedVersion,is_we_appreciate,is_sorry,total_hours,neg,neu,pos,compound
0,0,33.0,True,False,2.750000,0.0,1.000,0.000,0.0000
1,0,32.0,False,False,7.500000,0.0,1.000,0.000,0.0000
2,0,33.0,False,False,14.016667,0.0,0.417,0.583,0.4215
3,0,33.0,False,False,6.516667,0.0,0.694,0.306,0.4754
4,0,NaN,False,False,22.516667,0.0,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...
5841,0,NaN,False,False,0.733333,0.0,0.820,0.180,0.2960
5842,0,NaN,False,False,22.366667,0.0,1.000,0.000,0.0000
5843,0,32.0,False,False,1.466667,0.0,0.000,1.000,0.4215
5844,1,33.0,False,False,0.933333,0.0,1.000,0.000,0.0000


In [23]:
y_pred = model.predict(X_test)

In [24]:
#四捨五入して整数に変換
y_pred = np.round(y_pred).astype(int)
len(y_pred)

5846

In [26]:
#提出用ファイルを作成.2️列目を書き換える
df_submission = pd.read_csv("data/sample_submission.csv", header=None)
df_submission[1] = y_pred
df_submission.to_csv("sub.csv", index=False, header=False)